In [9]:
import numpy as np


# 语料

text = """
三体 世界 的 天空 没有 太阳 只有 三个 混乱 的 恒星
人类 文明 在 黑暗 森林 中 颤抖 每一次 呼吸 都是 赌注
面壁者 在 孤独 中 思考 破壁人 在 阴影 中 行动
舰队 穿越 宇宙 水滴 划破 寂静 智子 注视 着 地球
希望 与 毁灭 并存 战争 与 信仰 交织 宇宙 冷漠 而 宏大
""".split()

# 建立词表
vocab = list(set(text))
stoi = {w: i for i, w in enumerate(vocab)}
ids = np.array([stoi[w] for w in text])
V, D = len(vocab), 16  # V=词表大小，D=向量维度

# 初始化词向量
W_in = np.random.randn(V, D) * 0.1
W_out = np.random.randn(V, D) * 0.1

# Sigmoid 函数
def sigmoid(x):
    x = np.clip(x, -10, 10)
    return 1 / (1 + np.exp(-x))


#Skip-Gram + 负采样

lr = 0.02  # 学习率
for epoch in range(1000):
    loss = 0
    for i, c in enumerate(ids):
        # 上下文窗口 ±2
        ctx = [ids[j] for j in [i-2, i-1, i+1, i+2] if 0 <= j < len(ids)]
        for t in ctx:
            neg = np.random.randint(0, V, 4)  # 4个负样本
            for target, label in [(t, 1)] + [(n, 0) for n in neg]:
                dot = W_in[c] @ W_out[target]
                p = sigmoid(dot)
                grad = (label - p)
                # 更新中心词和上下文词的向量
                W_in[c] += lr * grad * W_out[target]
                W_out[target] += lr * grad * W_in[c]
                loss += -np.log(p + 1e-9) if label else -np.log(1 - p + 1e-9)
    if epoch % 50 == 0:
        print(f"Epoch {epoch}, loss = {loss:.2f}")


#查词语义相似度

def most_similar(word, topk=5):
    i = stoi[word]
    sim = W_in @ W_in[i]
    j = np.argsort(-sim)
    print(f"'{word}' 的相似词:", [vocab[k] for k in j[:topk]])


most_similar("人类")
most_similar("宇宙")
most_similar("面壁者")
most_similar("黑暗")


Epoch 0, loss = 727.97
Epoch 50, loss = 393.50
Epoch 100, loss = 232.07
Epoch 150, loss = 218.94
Epoch 200, loss = 219.11
Epoch 250, loss = 202.43
Epoch 300, loss = 194.18
Epoch 350, loss = 194.12
Epoch 400, loss = 208.76
Epoch 450, loss = 185.46
Epoch 500, loss = 189.30
Epoch 550, loss = 205.51
Epoch 600, loss = 209.47
Epoch 650, loss = 187.35
Epoch 700, loss = 204.50
Epoch 750, loss = 207.81
Epoch 800, loss = 201.10
Epoch 850, loss = 179.61
Epoch 900, loss = 209.09
Epoch 950, loss = 211.02
'人类' 的相似词: ['人类', '混乱', '黑暗', '文明', '恒星']
'宇宙' 的相似词: ['宇宙', '宏大', '行动', '寂静', '水滴']
'面壁者' 的相似词: ['面壁者', '思考', '呼吸', '赌注', '文明']
'黑暗' 的相似词: ['黑暗', '颤抖', '人类', '森林', '孤独']
